In [ ]:
#loading best model
from keras.models import load_model
import datetime
import numpy as np
from music21 import *

In [ ]:
model = load_model('/content/drive/MyDrive/ML_project/AMG/Models/instrumental.h5')

file1 = open("/content/drive/MyDrive/ML_project/AMG/xval_1.txt", 'r')
lines = file1.readlines()
array_string = ""
for line in lines:
    array_string += str(line)
file1.close()
array_string = array_string.replace("[","").replace("]","")
x_val = np.fromstring(array_string, dtype=int, sep=' ')
x_val = x_val.reshape(-1,32)

In [ ]:
import random
ind = np.random.randint(0,len(x_val)-1)

random_music = x_val[ind]
no_of_timesteps = 32

predictions=[]
for i in range(100):

    random_music = random_music.reshape(1,no_of_timesteps)

    prob  = model.predict(random_music)[0]
    y_pred= np.argmax(prob,axis=0)
    predictions.append(y_pred)

    random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
    random_music = random_music[1:]
    
print(predictions)

[14, 14, 84, 144, 14, 94, 46, 84, 46, 84, 46, 70, 80, 70, 46, 70, 46, 80, 80, 75, 46, 80, 80, 80, 80, 80, 80, 80, 137, 80, 80, 80, 42, 42, 42, 42, 137, 137, 42, 137, 42, 42, 42, 137, 42, 42, 137, 42, 137, 14, 137, 137, 137, 42, 137, 137, 137, 137, 130, 130, 130, 130, 62, 137, 14, 14, 137, 99, 137, 137, 137, 9, 137, 9, 62, 9, 14, 137, 58, 137, 76, 99, 76, 76, 14, 14, 76, 76, 76, 76, 76, 80, 28, 76, 76, 76, 76, 76, 76, 76]


In [ ]:
file2 = open("/content/drive/MyDrive/ML_project/AMG/unique_x_1.txt", 'r')
unique_x = file2.readline()
file2.close()

unique_x = unique_x.replace("'","").replace(" ","").replace("[","").replace("]","").split(",")
x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_x)) 
predicted_notes = [x_int_to_note[i] for i in predictions]

In [ ]:
def convert_to_midi(prediction_output):
   
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                
                cn=int(current_note)
                new_note = note.Note(cn)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
                
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            
        # pattern is a note
        else:
            
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 1
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='/content/drive/MyDrive/ML_project/AMG/Music_by_instrumental/'+str(datetime.datetime.now()).replace(' ','.')+'music.mid')


In [ ]:
convert_to_midi(predicted_notes)